# Getting started 
Welcome to the SCALE tutorial session! 

The following exercises aim to make you discover side-channel analysis techniques in Python. It is organised in several different part that aim to focus on the different topics covered by the book. In particular, the following subject will be covered

+ Appetizer: textbook correlation power analysis (CPA)
+ A more powerful threat: profiled attack
    + Identification of Points-Of-Interest (POIs)
    + A simple model: univariate gaussian template
    + Improving the model: bivariate gaussian template
    + More POI may mean more info: dimensionality reduction with Linear Discriminant Analysis
+ Which strategy is better: assessing the quality of a model
    + First approach: practical attack and key rank evaluation
    + Second approach: model quality and information theory metrics



## Target and Dataset Description

In the following exercises, you will work with practical measurements from an AES software implementation acquired from a Cortex-M4 target on a ChipWhisperer 308 evaluation board. The implementation uses a look-up table for the S-box layer. The measurements were taken using a Tektronix [CT1](https://www.tek.com/en/datasheet/current-probe/ct1-ct2-ct6) current probe from a [PICO6000E](https://www.picotech.com/oscilloscope/6000/picoscope-6000-overview?kit=6424E) oscilloscope with a sampling frequency of 625 MHz.

The dataset can be found [here](https://enigma.elen.ucl.ac.be/dsm/sharing/VMx9J3Xkc) and is organized as follows:

+ `long-traces`
+ `training0`
+ `validation0`
+ `validation1`
+ `validation2`
+ `validation3`
+ `validation4`
+ `ttest-kf`
+ `ttest-pf`

Where all directories contain the traces and the associated metadata of a single dataset. The dataset `training0` is for training purposes and consists of traces collected using plaintexts and keys randomly generated for each trace. Additionally, five validation datasets are provided. In contrast to the training dataset, these are collected using random plaintexts with the same fixed key (randomly generated) for each dataset. Knowing the key value used, these datasets are typically employed to assess the quality of a side-channel attack.
More specifically, the files `data.npz` are [Numpy archive](https://numpy.org/doc/stable/reference/generated/numpy.savez.html) files containing the following fields organized as 2D numpy arrays:

+ `traces`
    - int16 $\left[ nexec \times ns \right]$: collected traces for the $nexec$ executions measured. Each traces is composed of $ns$ measures.
+ `pts`
    - int8 $\left[ nexec \times 16 \right]$: plaintext used for each execution.
+ `ks`
    - int8 $\left[ nexec \times 16 \right]$: key used for each execution.
+ `cts`
    - int8 $\left[ nexec \times 16 \right]$: ciphertext used for each execution.

In practice, an adversary does not have access to the key when mounting a real attack. Nevertheless, as part of this training course, we are conducting a worst-case analysis of the security of an implementation. Therefore, we provide the evaluator with all the necessary information to assess the best possible attack.

## Get your hands dirty: how to use the dataset with Python

The following code snippet provides an example of how to use the dataset files, typically to load the measured traces or the associated metadata. To run it, you need to adjust the value of the variable `filepath` to the location of the file you want to load on your computer. Try running it to see what a power trace looks like! What can you already observe from the shapes of the traces?


In [ ]:
# Load the usefull packages
import numpy as np 
#%matplotlib inline
import matplotlib.pyplot as plt
from utils_scale import test_scale, utils_aes, utils_files

In [ ]:


# Here, the variable `filepath` containes the path to the `.npz` file from which
# you want to load the data. The path provided should be relative to the location of this notebook on your computer.
filepath = "scale-dataset/long-traces/data.npz"

# Open to file and load all the data fields from it. 
with open(filepath, 'rb') as f:
    coll = np.load(f, allow_pickle=True)
    traces = coll["traces"].astype(np.float64)
    pts = coll["pts"]
    ks = coll["ks"]
    cts = coll["cts"]

# Amount of traces to plot
am_traces = 10

# Create the figure and plot the loaded traces
f = plt.figure()
ax0 = f.add_subplot(1,1,1)
ax0.plot(traces[:am_traces,:].T)
ax0.set_xlabel("Time index")
ax0.set_ylabel("Power")
ax0.set_title(f"{am_traces} first power traces of the dataset")
plt.show()



The previous code snippet is provided as an example and allows you to verify that the datasets are accessible on your machine. For the rest of the exercises, we provide functions that will load the datasets for you, so you won't have to do it yourself. Next, we will focus on the 30.000 first time samples. Try to run the following code in order to see what it look like! What operations do you think the traces cover?

In [ ]:
filepath = "scale-dataset/training0/data.npz"

# Open to file and load all the data fields from it. 
with open(filepath, 'rb') as f:
    coll = np.load(f, allow_pickle=True)
    traces = coll["traces"].astype(np.float64)
    pts = coll["pts"]
    ks = coll["ks"]
    cts = coll["cts"]

# Amount of traces to plot
am_traces = 10

# Create the figure and plot the loaded traces
f = plt.figure()
ax0 = f.add_subplot(1,1,1)
ax0.plot(traces[:am_traces,:].T)
ax0.set_xlabel("Time index")
ax0.set_ylabel("Power")
ax0.set_title(f"{am_traces} first power traces of the dataset")
plt.show()

# Appetizer: Textbook Correlation Power Analysis (CPA)

In this section, we dig into side-channel analysis by jumping straight into an attack. The aim of this first exercise is to carry out the example attack presented in Chapter 1 of the book, commonly known as CPA.

As a reminder, this attack is a method that exploits the correlation between the practical measurements collected from an implementation execution and a model of these measurements. CPA generally uses an a-priori leakage model that assumesthat the power consumption of the target device is linearly correlated with it. A simple model proposed is the Hamming Weight model, which represents the leakage as the sum of the bits set to '1' for the values taken by a specific intermediate variable manipulated by the implementation (e.g., the values at the output of the S-box). This hypothesis is commonly found in the literature and is a reasonable approximation observed in practice for CMOS technology implementations.

To exploit such a model, Pearson’s correlation coefficient is a natural candidate. This statistic measures how well the leakages can be predicted by a linear function of the model values. The underlying idea is that the subkey hypothesis predicting the leakages best should be the correct one. This correlation is computed as

$$r_{xy} = \dfrac{ \mathsf{\hat E}(x^c y^c)}{\sqrt{\mathsf{\hat V}(x)\mathsf{\hat V}(y)}}$$

where $\mathsf{\hat E}$ and $\mathsf{\hat V}$ denote respectively the empirical average and variance, $ x $ represents the leakage values, and $ y $ represents the model values (e.g., the Hamming weight of the hypothetically manipulated values), while $x^c = x - \mathsf{\hat E}(x)$ (and similarly for $y^c$).

The first step of this exercise is to implement the computation of the correlation between measurements and their model, as defined below. Be aware that the amount of data can be very large; hence, the implementation of this function may be too memory-intensive for the machines used. In this case, you will need to process the data in blocks rather than processing the entire trace at once.


In [ ]:
def pearson_corr(x,y):
    """
    x: raw traces, as an np.array of shape (nb_traces, nb_samples) and type np.float64
    y: model, as an np.array of shape (nb_traces, nb_samples) and type np.int16
    return: the pearson coefficient of each samples as a np.array of shape (1,nb_samples)
    """
    #TODO
    
    ###ANSWER_START
    xc = x - np.mean(x, axis=0)
    yc = y - np.mean(y, axis=0)
    cov = np.mean(xc*yc, axis=0)
    
    x_std = np.std(x,axis=0)
    y_std = np.std(y,axis=0)
    
    return cov / (x_std * y_std)
    ###ANSWER_STOP

Next, we attempt to gain some intuition about CPA (and concurrently verify that you have properly implemented the correlation). To do this, we invite you to plot the correlation between the measurements and their estimates. Assuming the hypothesis holds, a correlation peak should appear at the time sample corresponding to the moment when the targeted intermediate value is actually manipulated by the device. See for yourself by implementing the functions `corr_HW_traces_outSB` defined below. What do you observe? Is the situation similar when the intermediate variables are not estimated properly (try modifying the value of `CST_MASK`)? Can you explain why? Before starting with your implementation, feel free to look at the `example_utils_aes()` function, which demonstrates how to use the resources available to you to compute the AES S-box or the Hamming Weight of a byte.

In [ ]:
from utils_scale.utils_aes import Sbox, HW
def example_utils_aes():
    # Use the following line to import the resources. 
    # In the context of the jupyter notebook, you can do it (outside a function) once and 
    # execute the code cell  

    # 'Sbox' and 'HW' are arrays that contains the values corresponding
    # to the byte value 'b' at the b-th index. See next
    
    # Sbox example
    sbox_byte_0x35 = Sbox[0x35]
    assert sbox_byte_0x35 == 0x96, "The result of Sbox[0x35] must be equal to 0x96"

    # Hamming Weight example
    HW_0xAA = HW[0xAA]
    HW_0x00 = HW[0x00]
    HW_0xFF = HW[0xFF]
    assert HW_0xAA == 4, "The HW 0xAA must be equal to 4"
    assert HW_0x00 == 0, "The HW 0xAA must be equal to 0"
    assert HW_0xFF == 8, "The HW 0xAA must be equal to 8"

    # Print SUCCESS if nothing went wrong (which should happen if you run this piece of code
    # without modifying it ;) )
    print("SUCCESS")

# Run to execute the examplary method
example_utils_aes()

In [ ]:
from utils_scale.utils_aes import Sbox, HW

def corr_HW_traces_outSB(traces, pts_bytes, k_bytes):
    '''
    traces: the power measurements performed for each encryption with a shape of (nb_traces, nb_samples)
    pts_bytes: a vector of bytes with shape (nb_traces, 1), corresponding to the plaintext bytes used by each executions for a single index. 
    k_bytes: a vector of bytes with shape (nb_traces, 1), corresponding to the key bytes used by each executions for a single index.
    return: the pearson correlation coefficient for each time sample of the trace as a numpy.array of shape (1,nb_samples)
    '''
    # TODO

    ###ANSWER_START
    # Compute state
    pAKSB = Sbox[pts_bytes ^ k_bytes]
    # Compute the HW
    HW_pAKSB = HW[pAKSB].reshape([len(pts_bytes),1])
    # Replicate the model for each time sample of the trace
    model_full = np.hstack([HW_pAKSB for _ in range(traces.shape[1])])
    # Compute correlation
    corrv = pearson_corr(traces,model_full)
    return corrv
    ###ANSWER_STOP

# Load utils to test the function
# Please ensure that you ran the previous code block before running this one.
# This code may take quite some time, especially if you did not implement the correlation using vector operation
index_byte_aes = 2
CST_MASK = 0x00 # Try to modify here

# Compute the correlation (you are not supposed to modify from here)
correlation = corr_HW_traces_outSB(traces, pts[:,index_byte_aes][:,np.newaxis], ks[:,index_byte_aes][:,np.newaxis]^CST_MASK)
test_scale.display_correlation(correlation, traces[0,:], index_byte_aes)

Next, we implement the attack itself. The most common CPA exploits leakages from intermediate values manipulated by the cryptographic implementation, whose value depends on both the key and known plaintexts (see KIS sensitive variable, Definition 3). As shown in the previous step, a good target when targeting AES appears to be the values manipulated after the substitution layer. The underlying idea is that the subkey hypothesis that predicts the leakages best should be the correct one.

However, making hypotheses about all the key bytes simultaneously can be computationally difficult. For this reason, a commonly used strategy is to follow a divide and conquer approach. This involves attacking different sub-parts of the key independently and combining the recovered bytes. In this exercise, we propose to follow such a strategy by targeting each byte of the key sequentially. To this end, the basic methodology can be summarized as follows:


+ For each key byte index $i$:
    + For each key candidate $k_{i}^* \in [0; 255]$:
        + Predict the intermediate values for each execution:
            + $x_i = p_i \oplus k_{i}^*$ when using the S-box input
            + $y_i = \text{Sbox}(x_i)$ when using the S-box output
        + Compute the Hamming Weight of the intermediate values, considered as the leakage model.
        + Compute the correlation between the model and the practical measurements provided.
    + Keep the value $k_{i}^*$ maximising the correlation as the predicted correct key byte.
 
To put this methodology into practice, we first ask you to perform the CPA using the leakage of the Sboxes output against a single byte by implementing the function `cpa_byte_out_sbox`. 

In [ ]:
def cpa_byte_out_sbox(pts_byte,traces):
    """
    pts: the plaintext byte used by each encryption performed.
    traces: the power measurements performed for each encryption.
    return: an np.array with the 256 key bytes candidates, ordered from the highest certainty to less one
    when performing the attack targeting the input of the sbox (so the value at index 0 is the most likely one).
    """
    # TODO

    ###ANSWER_START
    maxcorrs = 256*[0]
    for k in range(256):     
        # For the key guess, compute the result after the
        # substitution layer.
        pAKSB = Sbox[pts_byte ^ k]
        # Compute the model (HW) for each bytes
        HWpred = HW[pAKSB][:, np.newaxis]
        # Replicate the model for each time sample of the trace
        model_full = np.hstack([HWpred for _ in range(traces.shape[1])])
        # Compute correlation
        corrv = abs(pearson_corr(traces,model_full))
        # Recover max correlation coefficient
        maxcorrs[k] = np.max(corrv)
    # Sort key candidate per correlation coefficient
    idxsort = np.argsort(maxcorrs)
    return np.flip(idxsort)
    ###ANSWER_STOP

### Next, test your CPA implementation
byte_index = 4 # from 0 to 15
am_traces = 100

# Run your implementation
test_scale.test_cpa_single(byte_index, am_traces, pts, traces, ks, cpa_byte_out_sbox)

As the cherry on the top, the final step is to perform the CPA against the full key, by implementing the function `complete_cpa_out_sbox` defined next (hint: use the function `cpa_byte_out_sbox` that you just implemented). How many traces are required to recover the full key? CAUTION: this function may take time to proceed. 

In [ ]:
def complete_cpa_out_sbox(traces_atck, pts_atck):
    """
    traces_atck: a numpy array containing the traces considered for the attack
    pts_atck: a numpy array containing the plaintexts corresponding to the traces
    return: a list containing the recovered key bytes values.
    """
    # TODO
    
    ###ANSWER_START
    key_bytes_found = 16*[0]
    # Iterate over the indexes
    for i in tqdm.tqdm(range(16)):
        # Recover values for the byte index
        cpa_res = cpa_byte_out_sbox(pts_atck[:, i], traces_atck)      
        key_bytes_found[i] = cpa_res[0]
    return key_bytes_found
    ###ANSWER_STOP

# Here, run the test for the selected traces. 
# This execution can take some time ... be patient :) 
AM_TRACES_ATCK = [100] # Put more values here to perform the attack with different amount of trace
#test_scale.test_full_cpa(AM_TRACES_ATCK, traces, pts, ks, complete_cpa_out_sbox)

# Detection and mapping

The first section presents an attack to recover the key of a cryptographic implementation. However, the proposed method suffers from two main limitations. First, it is unprofiled, so its quality is highly dependent on the assumption made for the leakage model (i.e., HW in this case). Second, it is univariate (i.e., an attack takes advantage of a single measurement point). It follows from these two limitations that such an attack is sub-optimal compared to a (potentially multivariate) profiled attack. The latter will be the subject of the remaining sessions.

In general, profiled attacks are divided into two distinct phases: the profiling phase and the attack phase. In the first phase, an adversary is assumed to have access to a device similar to the one being attacked to create a leakage model. This model is then used in the attack phase to retrieve information about the value of the key manipulated by the targeted device.

Although the profiling phase makes it possible to reduce the number of traces needed to carry out the attack during the online phase, it comes with the difficulty of having to profile the device, which can be costly depending on the length of traces and the number of dimensions used. Faced with these limitations, detection and mapping methodologies emerge as a first strategy to reduce the profiling overheads. Their goal is to identify the time samples in the traces that carry information, allowing an adversary to focus data manipulation efforts only on these. Such time samples are depicted as Points Of Interest (POIs).

Next, we will deal with detection strategies commonly encountered in the literature. These have different advantages and disadvantages, and their use mainly depends on the opponent's objectives and constraints.

To identify POIs, we need to define what we are looking for when examining the traces. A hypothesis commonly used in the literature is to consider that leakage with respect to an intermediate variable can be characterized in two parts: the first is deterministic and depends on the target value, and the second is random and represents noise. Using the notation from Section 2.2 of the book, the $j$-th time sample of a trace with respect to the S-box output value denoted $z_i$ can be represented as:


$$l_j(z_i) = \delta_j (z_i) + r_j$$

where $\delta_j$ is the leakage deterministic part of the $j-$th time sample and $r_j$ is an additive noise.




## Profiled Correlation

For the first metric, we are interested in profiled correlation. In fact, you've already used a detection strategy: correlation! Remembering the appetizer section, the correlation peaks shown with the `display_correlation` function typically appear for the time sample where the intermediate state used to model the leakage is actually manipulated by the implementation.

However, the correlation values obtained are highly dependent on the leakage hypothesis used, and the Hamming Weight model may prove limiting compared to the more general additive model described above. Instead, a refinement of the model involves modeling the leakage associated with a value by the average of the measurements collected at a relevant time index.

Next, you have to implement the functions `profile_correlation` and `corr_prof_traces_outSB`. What relevant time index can you use? What do you observe in comparison to the HW correlation you did earlier?


In [ ]:
def profile_correlation(traces_p, classes_idx, time_index):
    """
    traces_p: the traces used for profiling, organised as a numpy array of float of shape [nb_traces, nb_samples]
    classes_idx: for each traces, the class index used associated. 
    time_index: the (relevant) time index used for profiling
    return: the estimation of the deterministic part of the leakage for each possible class and for each time sample. 
    Organised as a numpy array of shape (1, 256)
    """
    # TODO
    ###ANSWER_START
    # Compute the indexes of the traces belonging to each classes
    means = np.zeros(256)
    for i in range(256):
        means[i] = np.mean(traces_p[classes_idx==i,time_index],axis=0)
    return means[np.newaxis,:]
    ###ANSWER_STOP

def corr_prof_traces_outSB(traces_a, classes_idx, profiled_model):
    """
    traces_a: the traces used for correlation computation, organised as a numpy array of float of shape [nb_traces, nb_samples]
    classes_idx: for each traces, the class index used associated. 
    profiled_model: the profiled model to use for the correlation computation, numpy array of shape [1, 256]
    return: the pearson correlation coefficient for each time sample of the trace as a numpy.array of shape (1,nb_samples)
    """
    # TODO
    ###ANSWER_START
    # Format the model
    model_full = np.vstack([traces_a.shape[1]*[profiled_model[0,e]] for e in classes_idx])
    # Compute correlation
    corrv = pearson_corr(traces_a,model_full)
    return corrv
    ###ANSWER_STOP

# Load utils to test the function
# Please ensure that you ran the previous code block before running this one.
# This code may take quite some time, especially if you did not implement the correlation using vector operation
index_byte_aes = 2
relevant_time_index = 20023 # MODIFY here
amount_traces_profile = traces.shape[0]//2 # Try to change here

# Compute the correlation and display it (you are not supposed to modify from here)
SBout = Sbox[pts[:,index_byte_aes]^ks[:, index_byte_aes]]
delta = profile_correlation(traces[:amount_traces_profile, :], SBout[:amount_traces_profile], relevant_time_index)
correlation = corr_prof_traces_outSB(traces[amount_traces_profile:,:], SBout[amount_traces_profile:], delta)
test_scale.display_correlation(correlation, traces[0,:], index_byte_aes)


## Signal-to-Noise Ratio (SNR)

The second detection method we cover here is the Signal-to-Noise Ratio (SNR). As hinted by its name, it consists of computing the ratio between the signal and the noise levels, as detailed in Section 2.2.2 of the book.

On one hand, the signal can be intuitively seen as the variation between the deterministic part of the signal for each possible value taken by \( z_i \). However, characterizing this precisely in practice can be challenging. Instead, it is estimated by averaging several traces belonging to the same class (i.e., the traces collected for a given value taken by \( z_i \)), which is

$$\hat{\delta}_j(z_i) = \hat{\mathsf{E}}(L_j(z_i))$$. 

On the other hand, the noise is the variation that occurs in the measurements when the deterministic part is fixed. It can be estimated as the variance of the noise samples, defined as

$$r_j = l_j(z_i) - \hat{\delta}_j(z_i)$$

In our AES case study, we are interested in detecting the leakage of the independent 16 bytes at the S-box outputs. When targeting a single byte, the dataset composed of \( q \) traces is split into 256 different subsets (i.e., 1 per class) in order to estimate the means and the variances. Usually, the SNR is computed using intermediate variables following a uniform distribution in order to avoid biases, and the resulting subsets are composed of \( q/256 \) traces each. In such a case, the SNR is defined as (see Section 2.2.2, Eq. (5)):

$$\hat{\mathsf{SNR}}_q = \dfrac{\hat{\mathsf{Var}}_{256}(\hat{\mathsf{E}}_{q/256}(L_j(z_i)))}{\hat{\mathsf{E}}_{256}(\hat{\mathsf{Var}}_{q/256}(L_j(z_i)))}$$ 

Next, you have to implement the function `compute_byte_snr`. For this exercise, we consider only the SNR related to a single 8-bit intermediate variable, so you can consider that the class indexes provided span only in the range [0, 256]. Use the following piece of code to display the computed SNR for the different bytes at the output of the S-box layer. What do you observe? How do the results compare with the profiled correlation POI detection strategy? How do you explain it?


In [ ]:
def compute_byte_snr(traces, classes_idx):
    """
    traces: the traces on which to compute the SNR, organised as a numpy array of float of shape [nb_traces, nb_samples]
    classes_idx: for each traces, the class index used associated. 
    return: the SNR values computed for each time samples, as a numpy array of shape [nb_samples, 1]
    """
    # TODO

    ###ANSWER_START
    # Allocate memory
    snr_means = np.zeros([256, traces.shape[1]])
    snr_vars = np.zeros([256, traces.shape[1]])
    # Iterate over each class/byte values
    for i in range(256):
        # Compute the mean for the associated trace for each time samples
        snr_means[i] = np.mean(traces[classes_idx==i,:],axis=0)
        # Compute the variance of the samples
        snr_vars[i] = np.var(traces[classes_idx==i,:], axis=0)
    # Compute the SNR
    var_means = np.var(snr_means, axis=0)
    mean_vars = np.mean(snr_vars, axis=0)
    return var_means / mean_vars
    ###ANSWER_STOP

### Modify here to select display the SNR computed 
byte_index_targeted = 2 # Byte index to target

labels = Sbox[pts ^ ks]
test_scale.display_snr_sbox_output(traces, labels, byte_index_targeted, compute_byte_snr)

Have you ever heard about [SCALib](https://scalib.readthedocs.io/en/stable/)? It's an open-source library that aims to efficiently implement useful tools for side-channel analysis. More interestingly for our purposes, it implements the [SNR](https://scalib.readthedocs.io/en/stable/source/api/scalib.metrics.SNR.html#scalib.metrics.SNR). To validate your implementation, try computing the SNR using SCALib and implement the function `compute_byte_snr_scalib`.

In [ ]:
from scalib.metrics import SNR

def compute_byte_snr_scalib(traces, classes_idx):    
    """
    traces: the traces on which to compute the SNR, organised as a numpy array of float of shape [nb_traces, nb_samples]
    classes_idx: for each traces, the class index used associated. 
    return: the SNR values computed for each time samples, as a numpy array of shape [nb_samples, 1]
    """
    # TODO
    ###ANSWER_START
    snrobj = SNR(256,traces.shape[1],1)
    snrobj.fit_u(
        traces.astype(np.int16),
        classes_idx.astype(np.uint16)
    )
    snr_val = snrobj.get_snr()[0]
    return snr_val
    ###ANSWER_STOP

### Modify here to select display the SNR computed 
byte_index_targeted = 14 # Byte index to target
test_scale.display_snrs_sbox_output(traces, labels, byte_index_targeted, compute_byte_snr, compute_byte_snr_scalib)

#### The following code display the SNR for the selected SBoxes output using SCALib. 
bytes_indexes = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
test_scale.display_snr_sbox_output_SCALIB(traces, labels, bytes_indexes)

## Welch's T-test

The last detection method discussed is the Welch T-test, as developed in Section 2.2.3 of the book. As a reminder, it allows reducing the data complexity required for POI detection by decreasing the number of classes for which the signal must be estimated.

More specifically, the method consists of conducting a Welch T-test statistical test to observe if a difference of means exists between two sets of measurements. Here, the first setting we consider relies on the following sets:


- a *fixed* set of $q_{\text{f}}$ measurements, using a fixed plaintext $x_{\text{f}}$ and a fixed key $k_{\text{f}}$ for each trace.
- a *random* set of $q_{\text{r}}$ measurements, using random plaintext drawn uniformly and the fixed key $k_{\text{f}}$ for each trace.

Generally, a global dataset of $q$ measurements is collected such that $q_{\text{f}} \approx q_{\text{r}} \approx q/2$ and the latter is split in order to obtain the two datasets. From these, we can compute the following estimates for the $j$-th time sample:

$$
\begin{eqnarray}
\hat{\mu}_{\text{f}}(j) &=& \hat{\mathsf{E}}_{q_{\text{f}}}\left( L_j(x_{\text{f}}, k_{\text{f}})\right), \\
\hat{\mu}_{\text{r}}(j) &=& \hat{\mathsf{E}}_{q_{\text{r}}}\left( L_j(\$, k_{\text{f}}))\right), \\
\hat{\sigma}_{\text{f}}^2(j) &=& \hat{\mathsf{Var}}_{q_{\text{f}}}\left( L_j(x_{\text{f}}, k_{\text{f}})\right), \\
\hat{\sigma}_{\text{r}}^2(j) &=& \hat{\mathsf{Var}}_{q_{\text{r}}}\left( L_j(\$, k_{\text{f}}))\right),
\end{eqnarray}
$$

where $\$$ denotes a uniformly random value. Based on these, the T statistic for all time samples can be computed as:

$$\hat{\Delta}_{\frac{q}{2}}(j) = \dfrac{\hat{\mu}_{\text{f}}(j) - \hat{\mu}_{\text{r}}(j)}{\sqrt{\frac{\hat{\sigma}_{\text{f}}^2(j)}{q_{\text{f}}}} + \sqrt{\frac{\hat{\sigma}_{\text{r}}^2(j)}{q_{\text{r}}}}}$$

Finally, a popular rule-of-thumb assumes POIs to be detected if the t-statistic is higher than a threshold value of 4.5, which corresponds to a probability to reject the null hypothesis below $ 10^{-5} $ (i.e., that there is no difference of mean between the fixed and random leakage sets).

Next, we want you to identify interesting time samples using the T-test as a detection method in the setting described above. In practice, you have to implement the function `ttest_computation` defined next. Instead of implementing it yourself, don't hesitate to check the SCALib [documentation](https://scalib.readthedocs.io/en/stable/) for useful tools.
The provided example loads the datasets from `scale-dataset/ttest-kf` that have been generated considering the setting described above and displays the t-statistic you computed. What do you observe? How do you explain it?






In [ ]:
### Anwser 
from scalib.metrics import Ttest

def ttest_computation(traces, labels):
    """
    traces: matrix of shape (q, nb_samples) containing the traces of the first set used in the statistical test
    labels: vector of shape (q), containing the label of the class for each traces (i.e., either 0 or 1).
    return: a vector of shape [nb_samples,] containing the statistic values for each time samples. 
    """
    # TODO

    ###ANSWER_START
    ttest_obj = Ttest(traces.shape[1],d=1)
    ttest_obj.fit_u(traces, labels)
    return ttest_obj.get_ttest()[0]
    ###ANSWER_STOP


# Load the ttest dataset 
traces_tt, pts_tt, ks_tt, labels_tt = utils_files.load_ttest_dataset("scale-dataset/ttest-kf/data.npz")

# Compute the t-statistic and display the results
ttest_result = ttest_computation(traces_tt, labels_tt)
test_scale.display_ttest_result(ttest_result, traces_tt[0], title="Result k fixed ; pt fixed-vs-random")


Additionnally, we also provide a second dataset (under `scale-dataset/ttest-pf`) generated with the following setting:

- a *fixed* set of $q_{\text{f}}$ measurements, using a fixed plaintext $x_{\text{f}}$ and a fixed key $k_{\text{f}}$ for each trace.
- a *random* set of $q_{\text{r}}$ measurements, using the fixed plaintext $x_{\text{f}}$ and a random key drawn uniformly for each trace.

Try to compute the t-statistic in this configuration and compare it with the one you obtained before. What do you observe? How do you explain it? 

In [ ]:
# TODO
###ANSWER_START
traces_tt, pts_tt, ks_tt, labels_tt = utils_files.load_ttest_dataset("scale-dataset/ttest-pf/data.npz")

# Compute the t-statistic and display the results
ttest_result = ttest_computation(traces_tt, labels_tt)
test_scale.display_ttest_result(ttest_result, traces_tt[0], title="Result p fixed ; k fixed-vs-random")
###ANSWER_STOP

# Modeling and extraction

The previous section presented techniques for identifying POIs. In this section, we will focus on techniques for extracting information from these POIs in order to create a statistical model and use it to mount an attack. As a reminder, profiled attack are divided in two phases: the (offline) training phase, during which an adversary builds the model based on $q_{p}$ profiling traces, and the (online) attack phase during which he uses the model in order to recover sensitive information from $q_a$ (fresh) attack traces. Here, *Modeling* refers to the training phase while *extraction* refers to the attack phase in itself. 

In the profiled attack setting, we rely on two different datasets. The first is the training dataset contains measurements acquired using random plaintext and keys that are used to build the models while limiting bias. This one is typically acquired by the adversary during the offline phase by using the same device that he aims to attack. The second is the online dataset, which contains the measurements obtained when the targeted device performs executions using a fixed key value that it aims to recover (e.g., in the context of a real application). 
As part of our exercises, we use fixed-key datasets for which we know the value of the key. These datasets are called validation datasets, and are used to evaluate the practical performances of attacks carried out using the models you will build. 

The following piece of code allows to load the two datasets. In particular, we provide 5 different validation datasets that you can choose by modifying the variable `filepath_validation` that contains the path to the validation dataset file. For now, you can keep the default loading configuration. 

In [ ]:
# Filepath to the training dataset.
filepath_training = "scale-dataset/training0/data.npz"
# Filepath to the validation dataset.
filepath_validation = "scale-dataset/validation1/data.npz"

# Load all the data
training_traces, training_pts, training_ks, validation_traces, validation_pts, validation_ks = utils_files.load_datasets_profiled_setting(
    filepath_training,
    filepath_validation
)

utils_files.apply_permutation_dataset(training_traces, training_pts, training_ks)
utils_files.apply_permutation_dataset(validation_traces, validation_pts, validation_ks)


## Gaussian Template Attack

As a first exercise, we will carry out one of the most popular attacks: a Gaussian template attack. As detailed in the book in Section 2.3.1, this technique involves modeling the physical measurements as random variables that follow a Gaussian distribution, potentially distinct for all the different value manipulated. More into the details, the Probability Density Function (PDF) of the leakage trace $\boldsymbol{l}$ measured conditioned to the fact that the intermediate value $z_i$ is manipulated can be expressed as:

$$\mathsf{f}(\boldsymbol{l}|z_i) = \frac{1}{(2\pi)^{\frac{\tau}{2}}\left|\boldsymbol{\Sigma}_{z_i}\right|} \cdot \mathsf{exp}\left( -\frac{1}{2}\left(\boldsymbol{l} - \boldsymbol{\mu}_{z_i}\right)^{\intercal} \cdot \boldsymbol{\Sigma}_{z_i}^{-1} \cdot \left( \boldsymbol{l} - \boldsymbol{\mu}_{z_i}\right)\right)$$

where $\mu_{z_i}$ and $\boldsymbol{\Sigma}_{z_i}$ are respectively the means and the covariance matrix. The challenge of the profiling phase is therefore to estimate these (unknown) parameters of these distributions, which can be done by computing the empirical means and convariances:

$$
\begin{eqnarray}
    \hat{\boldsymbol{\mu}}_{z_i} &=& \hat{\mathsf{E}}_{\frac{q_p}{256}}\left( \boldsymbol{L}(z_i)\right), \\
    \hat{\boldsymbol{\Sigma}}_{z_i} &=& \hat{\mathsf{E}}_{\frac{q_p}{256}}\left( \left( \boldsymbol{L}(z_i) - \hat{\boldsymbol{\mu}}_{z_i}\right) \cdot \left( \boldsymbol{L}(z_i) - \hat{\boldsymbol{\mu}}_{z_i}\right)^{\intercal} \right).
\end{eqnarray}
$$

As a first step, you have to implement the function `POI_selection_SNR` that perform the POIs selection based on the SNR strategy. Don't hesitate to rely on a previously implemented function or SCALib! You can validate visually your implementation with the following code that displays the `amount_pois` POIs selected with your function for the `byte_index_targeted`-th Sbox output. 

In [ ]:
from scalib.metrics import SNR
def POI_selection_SNR(train_traces, byte_labels):
    """
    SNR based POIs selection
    train_traces: a matrix of shape (nb_traces, nb_samples) containing the training traces.
    byte_labels: a matrix of shape (nb_traces, n_p) containing the labels associated to each traces for n_p different intermediate variables.
    Returns: a matrix of shape (n_p, nb_samples) containing the POIs (i.e., time index) ordered from the most to the less interesting (i.e., from more to less SNR). 
    """
    # TODO
    ###ANSWER_START
    # Fetch dimension
    (nb_traces, nb_samples) = train_traces.shape
    n_p = byte_labels.shape[1]
    
    # Compute SNR on each time sample using SCALib
    snr_obj = SNR(256, nb_samples, np=n_p)
    snr_obj.fit_u(
        train_traces.astype(np.int16), 
        byte_labels.astype(np.uint16)
    )

    # Recover SNR values, sort and return
    snrs = snr_obj.get_snr()
    return np.flip(np.argsort(snrs,axis=1),axis=1)
    ###ANSWER_STOP

### 
amount_pois=1 # Try to modify here
byte_index_targeted=1 #Try to modify here

# Compute intermediate states 
pSB = Sbox[training_pts ^ training_ks]
# Compute the POIs based on your function
pois = POI_selection_SNR(training_traces, pSB)
# Display the selected POIS
test_scale.display_pois_TA(pois[byte_index_targeted, :amount_pois], validation_traces[0,:])

Next, we dig into the modeling phase and propose to start simple by considering univariate gaussian templates. That is, we will only exploit a single POI when building the model associated to an intermediate variable, which can however be different when building models for different intermediate variables (try to display the POIs for different intermediate states, e.g., different Sboxes if you don't see why). In the context of a univariate Gaussian distribution, the profilling phase boils down to estimating the empirical mean and the empirical variance such that:

$$
\begin{eqnarray}
\mathsf{f}(\boldsymbol{l}|z_i) \approx \hat{\mathsf{f}}(\boldsymbol{l}|z_i) &=& \mathcal{N}\left( \boldsymbol{l}| \hat{\mu}_{z_i}, \hat{\sigma}^2_{z_i} \right) \\
&=& \frac{1}{\hat{\sigma}_{z_i}\sqrt{2\pi}} \hspace{2mm}\exp\left( - \frac{\left( l - \hat{\mu}_{z_i}\right)^2}{2\hat{\sigma}_{z_i}^2}\right)
\end{eqnarray}
$$

As a second step, you have to compute $\hat{\mu}_{z_i}$ and $\hat{\sigma}_{z_i}$ by implementing the function `univariate_gaussian_models` defined next. At first, taking into account the small training data complexity, we invite you to implement a pooled variance estimation as described in Section 2.3.1. 

In [ ]:
def univariate_gaussian_models(train_traces, byte_labels, pois):
    """
    train_traces: a matrix of shape (nb_traces, nb_samples) containing the training traces
    byte_labels: a matrix of shape (nb_traces, n_p) containing the label associated to each trace for `n_p` different intermediate state.
    pois: a matrix of shape (n_p, 1) containing the single POI to keep for each of the `n_p` intermediate state
    return: (us, ss), where the matrices `us` and `ss` of shape (n_p, 256) contain respectively the mean and the standard deviation associated to each possible byte for each intermediate state.
    """
    # TODO
    ###ANSWER_START
    # Fetch dimension 
    n_p, npois = pois.shape
    
    # Allocate return value
    us = np.zeros([n_p,256])
    ss = np.zeros([n_p,256])
    
    # Iterate over every intermediate state and every byte values
    for si in range(n_p):
        # Allocate memory squared traces accumulator (used for pooled variance computation)
        sum_sq_center_traces = 0

        # Iterate over all the possible bytes values
        for b in range(256):
            # Compute class mean
            us[si,b] = np.mean(train_traces[byte_labels[:,si] == b,pois[si,0]])
            
            # Compute the centered traces
            ctraces = train_traces[byte_labels[:,si] == b, pois[si,0]] - us[si,b]
            
            # Accumulate the squared centered traces
            sum_sq_center_traces += np.sum(np.square(ctraces),axis=0)
            
        # Second, compute the pooled variance
        ss[si,:] = np.sqrt((sum_sq_center_traces / (train_traces.shape[0]-1)))
    # Return
    return (us, ss)
    ###ANSWER_STOP

### Display config
byte_index = 1 # In the range 0-15, byte index of the model to display 
byte_value_class = 0 # In the range 0-255, byte value for which to display the built model visual representation

#### TRAINING ####
# Re-use the POIs computed in the previous step: don't forget to run it ;) 
univariate_pois = pois[:,0][:,np.newaxis]
# Compute the models
models = univariate_gaussian_models(training_traces, pSB, univariate_pois)

# Display visual information about the model
test_scale.boxplot_univariate(
    training_traces, 
    pSB[:,byte_index], 
    univariate_pois[byte_index], 
    (models[0][byte_index],models[1][byte_index]), 
    byte_value_class)

Once these models (also known as templates) are created, it remains to see how they can lead to practical attacks. During the extraction phase, an adversary typically has access to the targeted device when it performs executions with a fixed key that he seeks to retrieve. To this end, he obtains a set of fresh measurements associated to different executions and can perform a maximum likelihood attack based on these measurements and the models he has constructed. More precisely, for each trace, we can estimate the probability that a specific key byte value is being manipulated. When targeting the output of the Sbox (as done in our exercise), the intermediate value $z_i$ used above can be expressed as $z_i=\mathsf{Sbox}(p_i \oplus k_i)$ and the aformentionned probability can be expressed as follows thanks to Bayes' law

$$
\begin{eqnarray}
\check{k}_i &=& \underset{k^*_i}{\mathsf{argmax}}\hspace{2mm}\hat{\mathsf{Pr}}\left( k_{i}^* | p_i, \boldsymbol{l}\right) \\
            &=& \underset{k^*_i}{\mathsf{argmax}}\hspace{2mm}\frac{\hat{\mathsf{f}}\left( \boldsymbol{l} | \mathsf{Sbox}\left(p_i \oplus k_{i}^*\right) \right) \cdot \mathsf{Pr}(k^*_i)}{\sum^{255}_{k'_i=0} \hat{\mathsf{f}}\left( \boldsymbol{l}|\mathsf{Sbox}\left(p_i \oplus k'_{i}\right)\right) \cdot \mathsf{Pr}\left( k'_i \right)}
\end{eqnarray}
$$

where $\mathsf{Pr}(x)$ is a shortcut for $\mathsf{Pr}\left(X =x \right)$ for conciseness.

Here, you have to implement the function `online_phase_TA_univariate` that implements the exploitation phase. For this exercise, we recommend working with vector operations and using the logarithms of probabilities rather than their direct values. Once implemented, the implemented example implements an attack using all the loaded validation set (which must succeed). 

In [ ]:
###ANSWER_START
def gaussian_pdf(xs, mean, std):
    coef = 1/(np.sqrt((std)*(2*np.pi)))
    return coef * np.exp(-(xs-mean)**2 / (2*(std**2)))
###ANSWER_STOP

###ANSWER_START
def log2Pr_y_given_l_univariate(ls, means, stds):
    # Allocate the probabilities matrix
    probas = np.zeros([ls.shape[0],256])
    for i in range(256):
        unorm = gaussian_pdf(ls, means[i], stds[i]).reshape([ls.shape[0],])
        probas[:, i] = unorm
        
    # Normalization and return
    return np.log2(probas/np.sum(probas, axis=1)[:,np.newaxis])
###ANSWER_STOP

def online_phase_TA_univariate(atck_traces, atck_pts, pois, models):
    """
    atck_traces: a matrix of shape (nb_traces, nb_samples) containing the attack traces
    atck_pts: a matrix of shape (nb_traces, n_p) containing the plaintext byte related to `n_p` sbox output. 
    pois: a matrix of shape (n_p, 1) containing the single POI to keep for each of the `n_p` sbox output
    models: (us, ss), where the matrices `us` and `ss` of shape (n_p, 256) contain respectively the mean and the standard deviation of the leakage associated to each possible byte for each intermediate state. 
    return: a matrix of size (n_p, 256), where the `n_p`-th row contains the probability of each key candidates.
    """
    # TODO
    ###ANSWER_START
    # Fetch dimension 
    n_p = pois.shape[0]
    
    # Allocate log-probabilities matrix
    lprobas = np.zeros([n_p, 256])
    
    # Iterate over every intermediate values
    for pi in range(n_p):
        # Compute the log probabilities of internal state given the leakage
        intern_lprobas = log2Pr_y_given_l_univariate(atck_traces[:,pois[pi]], models[0][pi], models[1][pi])
        
        # Iterate over each possible ks
        for ks in range(256):
            # Compute the internal state associated to the 'n_p'-th state under the key guess hypothesis
            yi = Sbox[atck_pts[:,pi] ^ ks]
            # Compute the log likelihood of the key guess
            lprobas[pi,ks] = np.sum(intern_lprobas[np.arange(atck_traces.shape[0]),yi])
            
    ## Normalization steps
    # Scaling to avoid numerical instabilities
    max_log = np.max(lprobas,axis=1)[:,np.newaxis]
    lprobas = lprobas - max_log 

    # Compute the sum of probas for each key guess
    sum_probas = np.sum(np.exp2(lprobas),axis=1)[:,np.newaxis]
    
    # Compute the normalized probabilities
    return np.exp2(lprobas - np.log2(sum_probas))

    
###ANSWER_STOP

############################
# Global attack flow: here, all your functions are put together to perform
# all the steps of the TA using all the 
############################

## The attack re-use the POIs and the models computed in the previous steps, don't forget to run these ;) 

#### ATTACK ####
print("Wait for the attack to finish...")
key_probs = online_phase_TA_univariate(validation_traces, validation_pts, univariate_pois, models)
for i in range(16):
    print("Found value 0x{:02x} at byte index {} [must be 0x{:02x}]".format(np.argmax(key_probs[i,:]),i,validation_ks[0,i]))


As a final step, we ask you to evaluate the performance of your attack with the data complexities. In particular, the following pieces of code relies on the functions you implemented, performs both the offline and the online phases and displays the probability of the correct key obtained for different attack complexities (depicted in black, similarly to the Figure 10 in Section 2.3.4). How many traces do you need to recover the value of a key byte with a high probability? Is it the same for every key byte? How does this complexity compares with the CPA you implemented before? How do you explain it? How does this complexity evolve when a smaller training data complexity is considered? 

In [ ]:
### Some configuration
# Specify here the path to the dataset files. 
filepath_training = "scale-dataset/training0/data.npz"
filepaths_validation = [
    "scale-dataset/validation0/data.npz",
    "scale-dataset/validation1/data.npz",
    "scale-dataset/validation2/data.npz",
    "scale-dataset/validation3/data.npz",
    "scale-dataset/validation4/data.npz",
]

# Dont hesitate to modify
complexities_attack = np.arange(1,25,1)
complexity_training = 16384

uni_TA_results = test_scale.test_univariate_TA_qa(
    filepath_training,
    filepaths_validation,
    complexity_training,
    complexities_attack,
    POI_selection_SNR,
    univariate_gaussian_models,
    online_phase_TA_univariate
)


In [ ]:
# Run the code here to display the results of the attacks
byte_indexes = [0,1,2,3,4,5,6,7,8,9,10] # Byte indexes for which to display attack results (must contain value in range 0-15).
test_scale.display_TA_qa_results(uni_TA_results, byte_indexes)

## Improving the Template Attack with a more advanced training phase
Let's now try to improve our attack by creating more complex models. As described in Sections 2.3.1 and 2.3.2 of the book, one straightforward way is to create multivariate models using multiple POIs in the traces and using dimensionality reduction techniques. 

Here, we will rely on the Linear Discriminant Analysis (LDA) to reduce the dimensionality prior to building the gaussian model. Instead of implementing all the computation by yourself, we invite you to rather rely on the SCALib library that already implements useful tools: the [LDAClassifier](https://scalib.readthedocs.io/en/stable/source/api/scalib.modeling.LDAClassifier.html#scalib.modeling.LDAClassifier) and the [MultiLDA](https://scalib.readthedocs.io/en/stable/source/api/scalib.modeling.MultiLDA.html) objects. Please carefully read the related documentation pages before continuing in order to understand how to use them! If you are interested into details about their practical implementation, don't hesitate go check their [source code](https://github.com/simple-crypto/SCALib) 

Next, you'll find some similarities with the last exercice since your goal is to re-implement a TA with a more complex training phase. In particular, your first task is to implement the functions `multivariate_gaussian_model_with_LDA` defined next. 

In [ ]:
from scalib.modeling import LDAClassifier, MultiLDA

def multivariate_gaussian_model_with_LDA(train_traces, byte_labels, pois, ndim_proj):
    """
    train_traces: a matrix of shape (nb_traces, nb_samples) containing the training traces
    byte_labels: a matrix of shape (nb_traces, n_p) containing the label associated to each trace for `n_p` different intermediate state.
    pois: a matrix of shape (n_p, npois) containing the POIs to keep for each of the `n_p` intermediate state
    return: a SCALib.MultiLDA instance containing the model for each of the `n_p` variables. In particular, `predict_proba` is ready to be used. 
    """
    # TODO
    ###ANSWER_START
    # Fetch configuration
    n_p, npois = pois.shape
    # Create the MLDA object from scalib
    mlda_obj = MultiLDA(n_p*[256],n_p*[ndim_proj],pois.tolist())
    # Fit the object
    mlda_obj.fit_u(train_traces.astype(np.int16), byte_labels.astype(np.uint16))
    # Solve the object
    mlda_obj.solve(False)
    # Return 
    return mlda_obj
    ###ANSWER_STOP

# CAUTION -> Re-use the POIs computed when you implemented POI_selection_SNR: don't forget to run it ;) 
### Configurations ###
amount_pois = 10 # Amount of POIs to keep
projection_ndim = 2 # Amount of dimensions after projection (must be smaller than the amount of pois kept).

#### TRAINING ####
# Re-use the POIs computed in the previous step: don't forget to run it ;) 
kept_pois = pois[:,:amount_pois]
# Compute the models
mlda_models = multivariate_gaussian_model_with_LDA(training_traces, pSB, kept_pois, projection_ndim)
print("Training finished!")

As you might expect at this stage, the second step is to implement the online phase on the basis of these models. To do this, you need to implement the function `online_phase_TA_multivariate_and_LDA` defined next. As for the univariate attack, the provided example implements an attack against all the key byte using all the attack traces that is expected to succeed.

In [ ]:
def online_phase_TA_multivariate_and_LDA(atck_traces, atck_pts, mlda_models):
    """
    atck_traces: a matrix of shape (nb_traces, nb_samples) containing the attack traces
    atck_pts: a matrix of shape (nb_traces, n_p) containing the plaintext byte related to `n_p` sbox output. 
    mlda_models: the SCALib.MultiLDA instance containing the model for each of the `n_p` variables.
    return: a matrix of size (n_p, 256), where the `n_p`-th row contains the probability of each key candidate.
    """
    # TODO
    ###ANSWER_START
     # Fetch dimension 
    n_p = pois.shape[0]
    # Allocate probabilities matrix
    lprobas = np.zeros([n_p, 256])
    # Compute the probabilities of internal state given the leakage
    intern_lprobas = np.log2(np.array(mlda_models.predict_proba(atck_traces.astype(np.int16))))
    # Iterate over every intermediate values
    for pi in range(n_p):
        # Iterate over each possible ks
        for ks in range(256):
            # Compute the internal state associated to the 'n_p'-th state
            yi = Sbox[atck_pts[:,pi] ^ ks]
            # Compute the log likelihood of the key guess
            lprobas[pi,ks] = np.sum(intern_lprobas[pi,np.arange(atck_traces.shape[0]),yi])
    ## Normalization steps
    # Scaling to avoid numerical instabilities
    max_log = np.max(lprobas,axis=1)[:,np.newaxis]
    lprobas = lprobas - max_log 

    # Compute the sum of probas for each key guess
    sum_probas = np.sum(np.exp2(lprobas),axis=1)[:,np.newaxis]
    
    # Compute the normalized probabilities
    return np.exp2(lprobas - np.log2(sum_probas))
    ###ANSWER_STOP

############################
# Global attack flow: here, all your functions are put together to perform
# all the steps of the TA using all the 
############################

## The attack re-use the POIs and the models computed in the previous steps, don't forget to run these ;) 

#### ATTACK ####
print("Wait for the attack to finish...")
key_probs = online_phase_TA_multivariate_and_LDA(validation_traces, validation_pts, mlda_models)

for i in range(16):
    print("Found value 0x{:02x} at byte index {} [must be 0x{:02x}]".format(np.argmax(key_probs[i,:]),i,validation_ks[0,i]))

At this stage, let's try to analyze the attack complexity achieved when using the (new) more complex model built with SCALib. For this puprose, here is a similar piece of code to what you encountered in the previous section that performs attack against the different validation dataset. Comparing to the performances achieved by the univarate Gaussian TA, what do you observe related to the data complexity? Optionally, do you observe any performance gain in term of elapsed time? 

In [ ]:
### CAUTION: the variables `filepath_training` and `filepaths_validation` 
# are re-used from the univariate attack. 

### Some configuration
# Dont hesitate to modify
complexities_attack = np.arange(1,25,1)
complexity_training = 16384

# Profiling configuration
amount_pois = 256 # [256] Amount of POIs to keep 
projection_ndim = 16 # [16] Amount of dimensions after projection (must be smaller than the amount of pois kept). 

multi_TA_results = test_scale.test_multivariate_TA_qa(
    filepath_training,
    filepaths_validation,
    amount_pois,
    projection_ndim,
    complexity_training,
    complexities_attack,
    POI_selection_SNR,
    multivariate_gaussian_model_with_LDA,
    online_phase_TA_multivariate_and_LDA
)


In [ ]:
# Run the code here to display the results of the attacks
byte_indexes = range(16) # Byte indexes for which to display attack results (must contain value in range 0-15).
test_scale.display_TA_qa_results(multi_TA_results, byte_indexes)

# Model quality assesment

When building a statistical model, a typical question that may raise is "how does my model compare to another one?", or put in another way, "How to assess the quality of my model?". Such question is crucial when it comes to choose hyperparameters values (e.g., amount of POIs vs numbers of projection dimension) or to see if a bigger training complexity can lead to substancial improvement of the attack taking into account the time and memory complexity of one evaluator's setup. in this Section, we will try to tackle the issue following two different approach: the rank estimation and the information theory metric computation. 

## Histogram-based rank estimation

It turns out that you already used a common technique to evaluate the quality of different models in the last sections: mounting practical attacks based on the latter and evaluate the attack data complexity achieved! In general, template attacks allow to compute the probabilities $\hat{\boldsymbol{p}}_{k_i}$ that contains the probabilities associated to the different subkeys. In our previous examples, $i \in [0:15]$ and $|\hat{\boldsymbol{p}}_{k_i}|=256$, where $|\cdot|$ denotes the cardinality operator. 

If the correct subkey is associated to the maximal probability for all subkey, then the attack is directly successful. However, the situation may be different in practice, with some subkeys potentially proving more difficult to find than others for a given data complexity. Key enumeration algorithm allows to recombine the $\hat{\boldsymbol{p}}_{k_i}$ in order to list all the full key candidates from the most likely to the least likely with the aim of then testing the candidates exhaustively. With such a list, the attacks can be compared based on the position of the correct key in the list (denoted as the rank of the key). 

Try it by yourself by implementing the function `key_rank_computation` defined next. The provide example allows you to visualize the calculated rank as a function of the attack complexity for all the validation sets. Considering a single byte, does the computed rank is coherent with your previous observations? How does the rank evolve when a worse model is considered (e.g., using less training traces)? Can you compute the rank for key of any size or are you limited to some bytes? How do you explain it? 

In [ ]:
def key_rank_computation(subkey_probs, correct_subkeys):
    """
    subkey_probs: matrix of shape (n_p, 256), containing the probabilities associated to the values of the `n_p` subkey bytes.
    correct_subkeys: a list of `n_p` elements, where the i-th element is the correct value of the i-th subkey. 
    return the rank of the full key (i.e., composed of all the subkeys). Must be in the range [0 ; 2^(8*n_p)]
    """
    # TODO
    ###ANSWER_START
    # Naive implementation: compute the probabilities of each key, then refind the location
    n_p = len(correct_subkeys)
    # Use a logarithmic representation of the probas
    subkey_lprobs = np.log(subkey_probs)
    # Allocate memory
    lprobs_keys = np.zeros(256**n_p)
    for i in range(256**n_p):
        # Recover the subkey bytes
        subkeys = [(i >> (8*j))&0xff for j in range(n_p)]
        # Compute the probas
        lprobs_keys[i] = np.sum(subkey_lprobs[np.arange(n_p),subkeys])
    # Sort the values
    sorted_keys = np.argsort(lprobs_keys)
    # Decimal representation of the correct_subkeys
    dec_correct_subkeys = np.sum([e<<(8*j) for j,e in enumerate(correct_subkeys)])
    rank = (256**n_p)-np.where(sorted_keys==dec_correct_subkeys)[0]
    return rank
    ###ANSWER_STOP

# Some configuration
byte_indexes = [0] # elements in range [0 ; 15], byte indexes to consider in order to compute the exact rank

### Plot the results
exacts_key_ranks_results = test_scale.test_key_rank_computation(multi_TA_results, byte_indexes, key_rank_computation)


Computing the exact rank of a key turns out to be challenging since this approach is bounded by the enumeration power of the evaluator. To tackle the issue, rank estimation techniques are efficient algorithm that have been proposed to approximate the rank in an "sufficiently enough" accurate manner. Next, we focus on the histogram-based rank estimation described in Section 2.5.1 and implemented by [SCALib](https://scalib.readthedocs.io/en/stable/source/api/scalib.postprocessing.rankestimation.html#module-scalib.postprocessing.rankestimation). 

Next, your goal is to implement the function `key_rank_approximation_scalib` that estimates the key rank using SCALib. The provided example allows you to compare your exhaustive computation with the approximated one. Is the approximation coherent? 


In [ ]:
from scalib.postprocessing import rank_accuracy
def key_rank_approximation_scalib(subkey_probs, correct_subkeys):
    """
    subkey_probs: matrix of shape (n_p, 256), containing the probabilities associated to the values of the `n_p` subkey bytes.
    correct_subkeys: a list of `n_p` elements, where the i-th element is the correct value of the i-th subkey. 
    return (rmin, r, rmax), where `r` is the approximated rank of the full key, `rmin` and `rmax` are respectively the minimal and maximal bounds.
    """
    #TODO
    ###ANSWER_START
    return rank_accuracy(-np.log(subkey_probs), correct_subkeys)
    ###ANSWER_STOP

# Some configuration
byte_indexes = [0] # elements in range [0 ; 15], byte indexes to consider in order to compute the exact rank
test_scale.display_rank_approx_vs_exact_rank(multi_TA_results, byte_indexes, exacts_key_ranks_results, key_rank_approximation_scalib)

As a final steps, the following piece of code compare the ranks obtained for the full 128-bit key between the univariate and the multivariate TA you implemented. Considering that you have access to a enumeration power of $2^{32}$, how much traces do you expect to recover the key with high probability using your univariate model? And with your multivariate model?  

In [ ]:
# Some configuration
byte_indexes = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
test_scale.display_rank_esti_full_key(uni_TA_results, multi_TA_results, byte_indexes, key_rank_approximation_scalib)

## Information Theory Metric 

Experimentally run several attacks with different models in order to compare their quality may turn out to be time- and data-intensive. In particular, this means collecting several independent validation sets and repeating attacks against them in order to gain confidence in the averaged performance metrics computed. 

Another way of evaluating the quality of a model is to rely on Information Theory (IT) metric. In particular, the Mutual Information (MI) between a sensitive intermediate variable and the practical leakage can be used to link the attack data complexity to its success rate, as detailed in Section 3.2.1 of the book. Intuitively, the more the leakage contains information about the targeted intermediate variable, the less traces are required to succeed. The MI is computed as follows:

$$\mathsf{MI}(V; \boldsymbol{L}) = \mathsf{H}(V) + \sum_{v\in V} \mathsf{Pr}(v)\cdot \int_{l\in L}\mathsf{f}(\boldsymbol{l}|v)\cdot \mathsf{log}_2\mathsf{Pr}(v|\boldsymbol{l})dl$$

where $\mathsf{H}(V)$ is the entropy of $V$ and $\mathsf{Pr}(v|\boldsymbol{l})$ can be computed thanks to Bayes law. However, as pointed out in Section 3.3.1, evaluating the exact MI turns out to be difficult since the true distribution of the leakage is unknown. Alternatively, we can estimate the amount of information that can be extracted from a given model using the Perceived Information (PI) and compare different models by comparing their PIs. PI is defined similarly to the MI with the difference that the value depending on the true leakage distribution are estimated, leading to its approximation given by 

$$\tilde{\mathsf{PI}}(V; \boldsymbol{L}) = \mathsf{H}(V) + \sum_{v\in V} \mathsf{Pr}(v) \cdot \sum_{\boldsymbol{l}\in \mathcal{L}_{t}(v)} \dfrac{1}{\left| \mathcal{L}_t(v) \right|} \cdot \mathsf{log}_2 \hat{\mathsf{Pr}}(v|\boldsymbol{l}) dl$$

where the hat notation is used to reflect the (potentially imperfect) estimation of the model, the tilde notation is used to reflect the estimation of the PI metric based on a set of test samples $\mathcal{L}_{\mathsf{t}}$, which is different from the set $\mathcal{L}_{\mathsf{p}}$ used for profiling.

Next, your goal is to implement the computation of the PI, provided a profilling set and a test set. In practice, are typically obtained by splitting a single data set in different parts. More particularly, you have to implement the functions `pi_uni_TA` and `pi_LDA_multi_TA` that compute the PI for the provided 8-bit variables using the univariate TA model and the multivariate with LDA TA model you did before. 

In [ ]:
def pi_uni_TA(traces_profile, labels_profiles, traces_test, labels_test):
    """
    traces_profile: matrix of shape (nb_profile, nb_samples) containing the training traces
    labels_profiles: matrix of shape (nb_profile, n_p) containing the labels associated to each traces for the `n_p` intermediate state
    traces_test: matrix of shape (nb_test, nb_samples) containing the test traces
    labels_test: matrix of shape (nb_test, n_p) containing the labels associated to each traces for the `n_p` intermediate state
    return: (pis, lprobs) where
        - `pis` is a vector of shape (n_p,) containing the PI computed for each intermediate state
        - `lprobas` is a matrix of shape (n_p, nb_test) containing the log2 proba used to compute the PI for each of the `n_p' and for each trace.
    """
    # TODO
    ###ANSWER_START
    # Fetch some dimensions
    nb_test, n_p = labels_test.shape
    
    # First of all, compute the models
    pois = POI_selection_SNR(traces_profile, labels_profiles)
    models = univariate_gaussian_models(traces_profile, labels_profiles, pois[:,0][:,np.newaxis])
    
    ## Second, compute the log probas for each traces
    # Allocate matrices
    lprobas = np.zeros([n_p, nb_test])
    pis = np.zeros(n_p)
    # Iterate over every intermediate states
    for pi in range(n_p):
        # Compute the probabilities of internal state given the leakage
        intern_lprobas = log2Pr_y_given_l_univariate(traces_test[:,pois[pi,0]], models[0][pi], models[1][pi])
        # Compute the average of the lprob
        lprobas[pi,:] = intern_lprobas[np.arange(nb_test), labels_test[:,pi]]
        avg_lprob = np.mean(lprobas[pi,:])
        # Compute the PI
        pis[pi] = 8 + avg_lprob
    # Return 
    return pis, lprobas
    ###ANSWER_STOP

def pi_LDA_multi_TA(traces_profile, labels_profiles, traces_test, labels_test, npois, ndim):
    """
    traces_profile: matrix of shape (nb_profile, nb_samples) containing the training traces
    labels_profiles: matrix of shape (nb_profile, n_p) containing the labels associated to each traces for the `n_p` intermediate state
    traces_test: matrix of shape (nb_test, nb_samples) containing the test traces
    labels_test: matrix of shape (nb_test, n_p) containing the labels associated to each traces for the `n_p` intermediate state
    npois: amount of pois used by the LDA
    ndim: amount of dimensions of the linear subspace projection. 
    return: (pis, lprobs) where
        - `pis` is a vector of shape (n_p,) containing the PI computed for each intermediate state
        - `lprobas` is a matrix of shape (n_p, nb_test) containing the log2 proba used to compute the PI for each of the `n_p' and for each trace.
    """
    # TODO
    ###ANSWER_START
    # Fetch some dimensions
    nb_test, n_p = labels_test.shape
    
    # First of all, compute the models
    pois = POI_selection_SNR(traces_profile, labels_profiles)
    models = multivariate_gaussian_model_with_LDA(traces_profile, labels_profiles, pois[:,:npois], ndim)
    
    ## Second, compute the log probas for each traces
    # Compute the probabilities of internal state given the leakage
    intern_lprobas = np.log2(models.predict_proba(traces_test.astype(np.int16)))
    # Compute the average of the lprob
    lprobas = np.zeros([n_p, nb_test])
    pis = np.zeros(n_p)
    for pi in range(n_p):
        lprobas[pi,:] = intern_lprobas[pi, np.arange(nb_test), labels_test[:,pi]]
        avg_lprob = np.mean(lprobas[pi,:])
        # Compute the PI
        pis[pi] = 8 + avg_lprob
    # Return 
    return pis, lprobas
    ###ANSWER_STOP


### Configurations for the LDA ###
amount_pois = 256 # Amount of POIs to keep
projection_ndim = 16 # Amount of dimensions after projection (must be smaller than the amount of pois kept).
amount_test_traces = 2048 # Amount of traces used for the test

qt_s = [3500, 4096, 6144, 7168, 8192, 14000]

# Compute the labels
pSB = Sbox[training_pts ^ training_ks]

# Compute the PI for the univariate model
results_pi_uni = test_scale.compute_pi_estimations(
    training_traces[:-amount_test_traces], 
    pSB[:-amount_test_traces], 
    training_traces[-amount_test_traces:], 
    pSB[-amount_test_traces:], 
    pi_uni_TA, 
    qt_s
)

# Compute PI for the multivariate model
wrap_pi_LDA_multi_TA = lambda a,b,c,d: pi_LDA_multi_TA(a, b, c, d, amount_pois, projection_ndim)
results_pi_multi = test_scale.compute_pi_estimations(
    training_traces[:-amount_test_traces], 
    pSB[:-amount_test_traces], 
    training_traces[-amount_test_traces:], 
    pSB[-amount_test_traces:], 
    wrap_pi_LDA_multi_TA, 
    qt_s
)


The following piece of code allows you to display the computet PI as well as a 2-sigma confidence interval. What do you observe? What can you conclude regarding the different models? 

In [ ]:
### Display
byte_indexes= [0,1] #range(16) # Elements in range [0; 15]. Bytes for which display the results.
test_scale.display_IT_results(
    byte_indexes, 
    [
        ("Univariate",results_pi_uni), 
        ("Multivariate", results_pi_multi)
    ],
)

To support the conclusion made in the previous point, a legitimate remaining question is whether the model can be further improved by using a higher profiling complexity than the one used. To this end, it is possible to evaluate the amount of information that remains to be learned by a model, as explained in Section 3.3.1, paragraph *Bounding the learnable information*. In particular, the supremum of the PI can be bounded using the Training Information (TI):

$$\tilde{\mathsf{TI}}(V; \boldsymbol{L}) = \mathsf{H}(V) + \sum_{v\in V}\mathsf{Pr}(v) \cdot \sum_{\boldsymbol{l}\in \mathcal{L}_{\mathsf{p}}(v)} \dfrac{1}{|\mathcal{L}_{\mathsf{p}}(v)|} \cdot \mathsf{log}_{2} \hat{\mathsf{Pr}}(v|\boldsymbol{l}) dl$$

which in fact corresponds to computing the PI over the training set used to build the model in an overfitted manner. Similarly to what you did for the PI, you following goal is to implement the functions `ti_uni_TA` and `ti_LDA_multi_TA` that compute the TI for the univariate and the multivariate models. 

In [ ]:
def ti_uni_TA(traces_profile, labels_profiles):
    """
    traces_profile: matrix of shape (nb_profile, nb_samples) containing the training traces
    labels_profiles: matrix of shape (nb_profile, n_p) containing the labels associated to each traces for the `n_p` intermediate state
    return: (tis, lprobs) where
        - `tis` is a vector of shape (n_p,) containing the TI computed for each intermediate state
        - `lprobas` is a matrix of shape (n_p, nb_test) containing the log2 proba used to compute the TI for each of the `n_p' and for each trace.
    """
    # TODO
    ###ANSWER_START
    return pi_uni_TA(traces_profile, labels_profiles, traces_profile, labels_profiles)
    ###ANSWER_STOP

def ti_LDA_multi_TA(traces_profile, labels_profiles, npois, ndim):
    """
    traces_profile: matrix of shape (nb_profile, nb_samples) containing the training traces
    labels_profiles: matrix of shape (nb_profile, n_p) containing the labels associated to each traces for the `n_p` intermediate state
    npois: amount of pois used by the LDA
    ndim: amount of dimensions of the linear subspace projection. 
    return: (tis, lprobs) where
        - `tis` is a vector of shape (n_p,) containing the TI computed for each intermediate state
        - `lprobas` is a matrix of shape (n_p, nb_test) containing the log2 proba used to compute the TI for each of the `n_p' and for each trace.
    """
    # TODO
    ###ANSWER_START
    return pi_LDA_multi_TA(traces_profile, labels_profiles, traces_profile, labels_profiles, npois, ndim)
    ###ANSWER_STOP

# Compute the TI with the same configuration than the one used for the PI computation
results_ti_uni = test_scale.compute_ti_estimations(
    training_traces[:-amount_test_traces], 
    pSB[:-amount_test_traces], 
    ti_uni_TA, 
    qt_s
)

# Compute TI for the multivariate model
wrap_ti_LDA_multi_TA = lambda a,b: ti_LDA_multi_TA(a, b, amount_pois, projection_ndim)
results_ti_multi = test_scale.compute_ti_estimations(
    training_traces[:-amount_test_traces], 
    pSB[:-amount_test_traces], 
    wrap_ti_LDA_multi_TA, 
    qt_s
)



The following piece of code display the results computed. What can you conclude? In particular, are the experimental results obtained by your attacks coherent with the IT metrics you computed? 

In [ ]:
### Display
byte_indexes= range(16) # Elements in range [0; 15]. Bytes for which display the results.
test_scale.display_IT_results(
    byte_indexes, 
    [
        ("Univariate",results_pi_uni), 
        ("Multivariate", results_pi_multi),
        ("Univariate",results_ti_uni),
        ("Multivariate", results_ti_multi)
    +]
)

## Summary question: Univariate Gaussian Template without pooling. 

You now have a set of tools and a good grasp of the concepts needed to answer the last question, which covers all the concepts we have covered in the various tutorials. In particular, how does a univariate TA model without pooling compare with the two models you have already used? What attack complexity can it achieve? For this question, we are volontarily not providing anwser template in order for you to go through the full evaluation process by yourself. However, don't hesitate to re-use function that you already implemented!


In [ ]:
# TODO
###ANSWER_START
def univariate_gaussian_models_nopool(train_traces, byte_labels, pois):
    """
    train_traces: a matrix of shape (nb_traces, nb_samples) containing the training traces
    byte_labels: a matrix of shape (nb_traces, n_p) containing the label associated to each trace for `n_p` different intermediate state.
    pois: a matrix of shape (n_p, 1) containing the single POI to keep for each of the `n_p` intermediate state
    return: (us, ss), where the matrices `us` and `ss` of shape (n_p, 256) contain respectively the mean and the standard deviation associated to each possible byte for each intermediate state.
    """
    # Fetch dimension 
    n_p, npois = pois.shape
    # Allocate return value
    us = np.zeros([n_p,256])
    ss = np.zeros([n_p,256])
    # Iterate over every intermediate state and every byte values
    for si in range(n_p):
        # First compute the mean
        for b in range(256):
            # Identify the traces corresponding manipulating the data
            #I = np.where(byte_labels[:,si] == b)[0]
            # Compute class mean
            us[si,b] = np.mean(train_traces[byte_labels[:,si] == b,pois[si,0]])
            ss[si,b] = np.std(train_traces[byte_labels[:,si] == b,pois[si,0]])
    # Return
    return (us, ss)

def pi_uni_TA_nopool(traces_profile, labels_profiles, traces_test, labels_test):
    nb_test, n_p = labels_test.shape
    ## First off all, compute the models
    pois = POI_selection_SNR(traces_profile, labels_profiles)
    models = univariate_gaussian_models_nopool(traces_profile, labels_profiles, pois[:,0][:,np.newaxis])
    ## Second, compute the log probas for each traces
    # Allocate matrices
    lprobas = np.zeros([n_p, nb_test])
    pis = np.zeros(n_p)
    # Iterate over every intermediate values
    for pi in range(n_p):
        # Compute the probabilities of internal state given the leakage
        intern_lprobas = log2Pr_y_given_l_univariate(traces_test[:,pois[pi,0]], models[0][pi], models[1][pi])
        # Compute the average of the lprob
        lprobas[pi,:] = intern_lprobas[np.arange(nb_test), labels_test[:,pi]]
        # Compute the average of the lprob
        avg_lprob = np.mean(lprobas[pi,:])
        # Compute the PI
        pis[pi] = 8 + avg_lprob
    # Return 
    return pis, lprobas

def ti_uni_TA_nopool(traces_profile, labels_profiles):
    return pi_uni_TA_nopool(traces_profile, labels_profiles,traces_profile, labels_profiles)

# Compute the PI using the non-pool
qt_s_nopool = [4096, 7000, 7168, 8192, 14000]

# Compute the PI for the univariate model
results_pi_uni_pool = test_scale.compute_pi_estimations(
    training_traces[:-amount_test_traces], 
    pSB[:-amount_test_traces], 
    training_traces[-amount_test_traces:], 
    pSB[-amount_test_traces:], 
    pi_uni_TA_nopool, 
    qt_s_nopool
)
# Compute the TI with the same configuration than the one used for the PI computation
results_ti_uni_pool = test_scale.compute_ti_estimations(
    training_traces[:-amount_test_traces], 
    pSB[:-amount_test_traces], 
    ti_uni_TA_nopool, 
    qt_s_nopool
)
###ANSWER_STOP

In [ ]:
# TODO
###ANSWER_START
### Display
byte_indexes= range(16) # Elements in range [0; 15]. Bytes for which display the results.
test_scale.display_IT_results(
    byte_indexes, 
    [
        ("Uni Pool",results_pi_uni), 
        #("Multivariate", results_pi_multi),
        ("Uni Pool",results_ti_uni),
        ("Multivariate", results_ti_multi),
        ("Uni NoPool", results_pi_uni_pool),
        ("Uni NoPool", results_ti_uni_pool),
    ]
)
###ANSWER_STOP

## Going further: Advanced attack strategy with SASCA.
In this exercise, we will try to improve the performance of the multivariate attack even further by using different intermediate values that can provide information. More particularly, you will implement an attack relying on the Soft-Analytical Side-Channel Attack strategy (as detailled in Section 2.4.1 of the book). Once again, we are going to rely on the tools implemented by SCALib in order to facilitate the implementation: it turns out that the belief propagation algorithm consisting in performing multiple passes of equations (23) and (24) of the book can be (relatively) easily implemented as shown by the example presented in the [documentation](https://scalib.readthedocs.io/en/stable/source/api/scalib.attacks.FactorGraph.html#scalib.attacks.FactorGraph).

Next, you have to replicate the multivariate attack taking advantage of the Sbox outputs, but relying on a SASCA rather than the `online_phase_TA_multivariate_and_LDA` function that you have implemented. Don't hesitate to strongly rely on the documentation example! Additionnally, you might want to have a look at the documentation of [bp_acyclic](https://scalib.readthedocs.io/en/stable/source/api/scalib.attacks.BPState.html#scalib.attacks.BPState.bp_acyclic). 

In [ ]:
from scalib.attacks import FactorGraph, BPState

MIXCOLUMNS_INDEX = [
    [0,5,10,15],
    [4,9,14,3],
    [8,13,2,7],
    [12,1,6,11]
]

XTIME = np.array([
    0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 
    32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 
    64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 
    96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 
    128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 
    160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 
    192, 194, 196, 198, 200, 202, 204, 206, 208, 210, 212, 214, 216, 218, 220, 222, 
    224, 226, 228, 230, 232, 234, 236, 238, 240, 242, 244, 246, 248, 250, 252, 254, 
    27, 25, 31, 29, 19, 17, 23, 21, 11, 9, 15, 13, 3, 1, 7, 5, 
    59, 57, 63, 61, 51, 49, 55, 53, 43, 41, 47, 45, 35, 33, 39, 37, 
    91, 89, 95, 93, 83, 81, 87, 85, 75, 73, 79, 77, 67, 65, 71, 69, 
    123, 121, 127, 125, 115, 113, 119, 117, 107, 105, 111, 109, 99, 97, 103, 101, 
    155, 153, 159, 157, 147, 145, 151, 149, 139, 137, 143, 141, 131, 129, 135, 133, 
    187, 185, 191, 189, 179, 177, 183, 181, 171, 169, 175, 173, 163, 161, 167, 165, 
    219, 217, 223, 221, 211, 209, 215, 213, 203, 201, 207, 205, 195, 193, 199, 197, 
    251, 249, 255, 253, 243, 241, 247, 245, 235, 233, 239, 237, 227, 225, 231, 229, 
],dtype=np.uint8)

def times0x03(values):
    return XTIME[values] ^ values

def MC_tmp_lables(pSB, idx):
    mc1_0 = XTIME[pSB[:,idx[0]]] ^ pSB[:,idx[2]] ^ pSB[:, idx[3]] ^ times0x03(pSB[:,idx[1]])
    mc1_1 = pSB[:,idx[0]] ^ XTIME[pSB[:,idx[1]]] ^ pSB[:, idx[3]] ^ times0x03(pSB[:,idx[2]])
    mc1_2 = pSB[:,idx[0]] ^ pSB[:,idx[1]] ^ XTIME[pSB[:, idx[2]]] ^ times0x03(pSB[:,idx[3]])
    mc1_3 = pSB[:, idx[1]] ^ pSB[:, idx[2]] ^ XTIME[pSB[:,idx[3]]] ^ times0x03(pSB[:,idx[0]])

    return np.hstack([
        mc1_0[:,np.newaxis],
        mc1_1[:,np.newaxis],
        mc1_2[:,np.newaxis],
        mc1_3[:,np.newaxis]
    ])
    
# LAST MINUTE FIX: the two following functions must be copy-pasted from the cell below ... Sorry for the inconvenience :)
def apply_key_schedule(keys, rcon):
    rkeys = np.zeros(keys.shape, dtype=keys.dtype)
    # lcols 
    lcols = keys[:, 12:]
    lcols = Sbox[lcols]
    # Rotate Sbox
    tmp = np.zeros(lcols.shape, dtype=lcols.dtype)
    tmp[:,:3] = lcols[:, 1:]
    tmp[:, 3] = lcols[:, 0]
    # Add RCON
    tmp[:, 0] ^= rcon
    # Create first new col
    rkeys[:, :4] = tmp ^ keys[: , :4]
    # XOR the remaining column
    for i in range(1,4):
        rkeys[:,4*i:4*(i+1)] = keys[:,4*i:4*(i+1)] ^ rkeys[:, 4*(i-1):4*i]
    return rkeys

def RK_round(keys, nrounds):
    RCON = [0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36]
    tmp = keys.copy()
    for i in range(nrounds):
        tmp = apply_key_schedule(tmp, RCON[i])
    return tmp

def labels_intermediate(pts, ks):
    # Interesting labels: S-box output, the key itself, the next round key.
    # For the first exercice, only the sbox output are usefull. 
    rkeys = RK_round(ks, 1)
    sboxes = Sbox[pts ^ ks]
    mcs = np.hstack(
        [MC_tmp_lables(sboxes, MIXCOLUMNS_INDEX[i]) for i in range(4)]
         )

    return np.hstack([
        sboxes,
        ks,
        rkeys,
        mcs
    ])

# Compute labels
training_labels = labels_intermediate(training_pts, training_ks)

# Fit the models 
npois = 256
ndim = 16
pois_sasca = POI_selection_SNR(training_traces, training_labels)
models = multivariate_gaussian_model_with_LDA(training_traces, training_labels, pois_sasca[:,:npois], ndim)

# Compute SNR on each time sample using SCALib
snr_obj = SNR(256, training_traces.shape[1], np=training_labels.shape[1])
snr_obj.fit_u(
    training_traces.astype(np.int16), 
    training_labels.astype(np.uint16)
)
snrvs = snr_obj.get_snr()

f = plt.figure()
ax0 = f.add_subplot(2,1,1)
ax1 = f.add_subplot(2,1,2)

ax0.plot(training_traces[0])
mcs_v = 5
ax1.plot(snrvs[48+mcs_v])


# TODO: use FactorGraph and BPState to run acyclic belief propagation on each of the key bytes.
# The graph should contain the plaintext, the key, and the sbox.
# Then, compute the key rank.
###ANSWER_START

# GRAPH
graph_desc = '''
    NC 256
    TABLE Sbox
    VAR SINGLE k
    PUB MULTI p
    VAR MULTI x
    VAR MULTI y
    PROPERTY x = k ^ p
    PROPERTY y = Sbox[x]
    '''

graph = FactorGraph(graph_desc, {"Sbox": Sbox.astype(np.uint32)})
ntr = 1

evidence = models.predict_proba(validation_traces[:ntr,:].astype(np.int16))

scores = np.zeros((16,256))

for i in range(16):
    bp = BPState(graph, ntr, {"p": validation_pts[:ntr,i].astype(np.uint32)})    
    bp.set_evidence("y", evidence[i])
    bp.set_evidence("k", evidence[16+i][0,:])
    bp.bp_acyclic("k")
    k_distri = bp.get_distribution("k")
    scores[i] = -np.log(k_distri)
    print(f"byte {i: 2}, value: {validation_ks[0,i]}, guesses:", np.argsort(k_distri)[-15:][::-1])

rk  = rank_accuracy(scores, validation_ks[0])
print('rank est (log2):', np.log2(rk[1]))

###ANSWER_STOP

Next, we try to improve the SASCA by using more information source. That is, we will add some node in our factor graph, in order to take advantage of additional sources of leakage! The following pieces of code allow to display the SNR related to the round keys and implement the advanced SASCA using the leakages of the key and the key scheduling operation in addition to the S-boxes outputs. Don't hesitate to play with it in order to understand the effects of this additional information. What do you observe? 

In [ ]:
# Disclaimer: hastily-written code, do not copy its style ;)

# Example of SNR against the key schedule
from utils_scale.utils_aes import Sbox, HW
from scalib.metrics import SNR



# Compute the Round keys
RKA_idx = 1
RKB_idx = 5
rkeys_A = RK_round(ks, RKA_idx)
rkeys_B = RK_round(ks, RKB_idx)

byte_idx_0 = 0
byte_idx_1 = 15

# Compute the SNR for the round key with SCALib
snrobj = SNR(256,traces.shape[1],2*rkeys_A.shape[1])
snrobj.fit_u(
    traces.astype(np.int16),
    np.hstack([
        rkeys_A.astype(np.uint16),
        rkeys_B.astype(np.uint16)
    ])
)
snrvs = snrobj.get_snr()



####### Display everything
f = plt.figure()
ax0 = f.add_subplot(3,1,1)
ax1 = f.add_subplot(3,1,2)
ax2 = f.add_subplot(3,1,3)
ax0.plot(traces[0])
ax1.plot(snrvs[byte_idx_0], color="xkcd:red", label=r"$RK_{{{}}}^{{{}}}$".format(RKA_idx, byte_idx_0))
ax1.plot(snrvs[byte_idx_1], color="xkcd:green", label=r"$RK_{{{}}}^{{{}}}$".format(RKA_idx, byte_idx_1))
ax2.plot(snrvs[16+byte_idx_0], color="xkcd:red", label=r"$RK_{{{}}}^{{{}}}$".format(RKB_idx, byte_idx_0))
ax2.plot(snrvs[16+byte_idx_1], color="xkcd:green", label=r"$RK_{{{}}}^{{{}}}$".format(RKB_idx, byte_idx_1))
ax0.set_title("Trace")
ax1.set_title("SNR for RK of round {}".format(RKA_idx))
ax2.set_title("SNR for RK of round {}".format(RKB_idx))
ax1.set_ylabel("SNR")
ax2.set_ylabel("SNR")
ax2.set_xlabel("time index")
ax1.legend()
ax2.legend()
f.tight_layout()

In [ ]:
# GRAPH



def generate_graph_lines(c, cindex):
    lines = []
    for e in range(4):
        lines.append(f'VAR MULTI mc1_{4*cindex+e}')
    # Generate the lines 
    lines.append(f"PROPERTY mc1_{4*cindex} = xt1_{c[0]} ^ y_{c[2]} ^ y_{c[3]} ^ xt1_{c[1]} ^ y_{c[1]}")
    lines.append(f"PROPERTY mc1_{4*cindex+1} = y_{c[0]} ^ xt1_{c[1]} ^ y_{c[3]} ^ xt1_{c[2]} ^ y_{c[2]}")
    lines.append(f"PROPERTY mc1_{4*cindex+2} = y_{c[0]} ^ y_{c[1]} ^ xt1_{c[2]} ^ xt1_{c[3]} ^ y_{c[3]}")
    lines.append(f"PROPERTY mc1_{4*cindex+3} = y_{c[1]} ^ y_{c[2]} ^ xt1_{c[3]} ^ xt1_{c[0]} ^ y_{c[0]}")
    return lines

graph_lines = [
    'NC 256',
    'TABLE Sbox',
    'TABLE xtime',
    *(f'VAR SINGLE k_{i}' for i in range(16)),
    *(f'PUB MULTI p_{i}' for i in range(16)),
    *(f'VAR MULTI x_{i}' for i in range(16)),
    *(f'VAR MULTI y_{i}' for i in range(16)),
    *(f'VAR SINGLE kr1_{i}' for i in range(16)),
    *(f'VAR MULTI xt1_{i}' for i in range(16)),
    'PUB SINGLE cst1',
    *(f'VAR SINGLE sb_kr0_{i}' for i in range(12, 16)),
    *(f'PROPERTY x_{i} = k_{i} ^ p_{i}' for i in range(16)),
    *(f'PROPERTY y_{i} = Sbox[x_{i}]' for i in range(16)),
    *(f'PROPERTY sb_kr0_{i} = Sbox[k_{i}]' for i in range(12, 16)),
    'PROPERTY kr1_0 = k_0 ^ sb_kr0_13 ^ cst1',
    'PROPERTY kr1_1 = k_1 ^ sb_kr0_14',
    'PROPERTY kr1_2 = k_2 ^ sb_kr0_15',
    'PROPERTY kr1_3 = k_3 ^ sb_kr0_12',
    *(f'PROPERTY kr1_{i} = k_{i} ^ kr1_{i-4}' for i in range(4,12)),
    *(f'PROPERTY xt1_{i} = xtime[y_{i}]' for i in range(16)), 
]

amount_cols_mc = 4
for i in range(4):
    lines_mc = generate_graph_lines(MIXCOLUMNS_INDEX[i], i)
    graph_lines += lines_mc

graph_desc = '\n'.join(graph_lines)

graph = FactorGraph(graph_desc, {"Sbox": Sbox.astype(np.uint32), "xtime": XTIME.astype(np.uint32)})
tr_offset = 0
ntr = 1

bp = BPState(graph, ntr, {f"p_{i}": validation_pts[tr_offset:tr_offset+ntr,i].astype(np.uint32) for i in range(16)} | {'cst1': 0x1})
evidence = models.predict_proba(validation_traces[tr_offset:tr_offset+ntr,:].astype(np.int16))

for i in range(16):
    bp.set_evidence(f"y_{i}", evidence[i])
    #bp.set_evidence(f"k_{i}", evidence[16+i][0,:])
    #bp.set_evidence(f"kr1_{i}", evidence[32+i][0,:])
for i in range(4*amount_cols_mc):
    a = 0
    bp.set_evidence(f"mc1_{i}", evidence[48+i])


scores = np.zeros((16,256))
for i in range(16):
    bp.bp_loopy(it=4, initialize_states=True)#.bp_acyclic(f"k_{i}")
    #bp.bp_acyclic(f"k_{i}")
    k_distri = bp.get_distribution(f"k_{i}")
    scores[i] = -np.log(k_distri)
    print(f"byte {i: 2}, value: {validation_ks[0,i]}, guesses:", np.argsort(k_distri)[-15:][::-1])

rk  = rank_accuracy(scores, validation_ks[0])
print('rank est (log2):', np.log2(rk[1]))